In [3]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

import os

habsosdata = '/content/drive/My Drive/Team15-Biointerphase/Dataset/(final)cleaned_habsos_data_v3.csv'
argosdata = '/content/drive/My Drive/Team15-Biointerphase/Dataset/ARGO_FloatR.csv'

if os.path.exists(habsosdata):
    print("exists")
else:
    print("file not found")

if os.path.exists(argosdata):
    print("exists")
else:
    print("file not found")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
exists
exists


In [ ]:
import pandas as pd

# Load the first dataset with default settings
habsos = pd.read_csv(habsosdata)

# Define the column types for the second dataset to save memory
dtype = {
    'latitude': 'float32',
    'longitude': 'float32'
}

# Load the second dataset in chunks
chunk_size = 50000  # You can adjust this size as necessary
chunks = []

for chunk in pd.read_csv(argosdata, header=0, skiprows=[1], dtype=dtype, chunksize=chunk_size):
    # Rename columns as needed
    chunk.rename(columns={'time': 'SAMPLE_DATE'}, inplace=True)
    chunk.rename(columns={'latitude': 'LATITUDE'}, inplace=True)
    chunk.rename(columns={'longitude': 'LONGITUDE'}, inplace=True)

    # Convert SAMPLE_DATE to datetime if it wasn't handled in dtype
    chunk['SAMPLE_DATE'] = pd.to_datetime(chunk['SAMPLE_DATE'])

    # Append the chunk to the list
    chunks.append(chunk)

# Concatenate all chunks into a single DataFrame
second_dataset = pd.concat(chunks, ignore_index=True)

# Convert SAMPLE_DATE in habsos to naive datetime
habsos['SAMPLE_DATE'] = pd.to_datetime(habsos['SAMPLE_DATE']).dt.tz_localize(None)

# Convert SAMPLE_DATE in second_dataset to naive datetime (if it has timezone)
second_dataset['SAMPLE_DATE'] = second_dataset['SAMPLE_DATE'].dt.tz_localize(None)

# Merge the two datasets based on 'SAMPLE_DATE', 'latitude', and 'longitude'
merged_data = pd.merge(
    habsos,
    second_dataset,
    on=['SAMPLE_DATE', 'LATITUDE', 'LONGITUDE'],
    how='left',  # Use 'left' join to keep all rows from habsos
    suffixes=('_habsos', '_second')
)

# Save the merged data to a new CSV file
merged_data.to_csv('merged_output.csv', index=False)

print("Merging complete. The merged data is saved in 'merged_output.csv'.")
